### 数据收集


In [3]:
import pandas as pd
import pymysql
db = pymysql.connect('mysql-backup.dian.so','yihui','GMYAvsgzE9fRRC1r','oss_view') # 打开数据库连接
sql='''
select e.id as '用户id', nick_name as '花名',
e.sex as '性别',
education_first as '学历',
r.birth_date as '生日',
e.level_id as '级别' 
,recruite_channel as '来源' ,
e.department_id  as '部门id-1',
(
select name from agent_department ad where id=e.department_id
)as '部门名称-1',
(
select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=e.department_id 
)
)as '部门名称-2',
(
select id from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=e.department_id 
)
)as '部门id-2',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=e.department_id 
  )
 )
)as '部门名称-3',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=e.department_id 
    )
  )
 )
)as '部门名称-4',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=e.department_id 

          )
    )
  )
 )
)as '部门名称-5',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=e.department_id 
                      )

          )
    )
  )
 )
)as '部门名称-6',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                                            select parent_id from agent_department ad4 where id=e.department_id 

                      )
                      )

          )
    )
  )
 )
)as '部门名称-7'

from  hr_resume as r left JOIN  agent_employee as e
on e.id=r.employee_id
where  real_status=1 and status=0 and agent_id=1
order by e.id desc
'''

user=pd.read_sql(sql=sql,con=db)
db.close()

In [4]:
user.head(3)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7
0,24397,欧科,1,大专,1996-02-25,2,None,1999,银川BDM_丽莎,银川,1970,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None
1,24395,快驰,1,大专,1997-02-17,2,None,3330,上饶,华中大区,1977,仓库管理部,仓储部,供应链中心,杭州小电科技股份有限公司,None
2,24394,甄帅,1,高中,1995-09-02,2,None,2108,重庆BDM_欧成,重庆二区,1966,重昆分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司


### 数据处理

In [5]:
"""
时间转换成年代
"""
def get_birth_day(birth_date):
    dt95='1995-01-01'
    dt90='1990-01-01'
    dt85='1985-01-01'
    dt80='1980-01-01'

    birth=str(birth_date)
    if birth>dt95:
        return '95后'
    if birth<dt95 and birth>dt90:
        return '90-95'
    if birth>dt85 and birth<dt90:
        return '85-90'
    if birth>dt80 and birth<dt85:
        return '80-85'
    if birth<dt80:
        return '70后'
    return '未知'

"""
将性别标记位转换成文字
"""
def get_sex(sex):
    if sex==1:
        return '男'
    else:
        return '女'


In [6]:
user['性别']=user.apply(lambda x: get_sex(x['性别']),axis=1)


In [8]:
user['年龄段']=user.apply(lambda x: get_birth_day(x['生日']),axis=1)


In [9]:
user.head(20)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7,年龄段
0,24397,欧科,男,大专,1996-02-25,2,None,1999,银川BDM_丽莎,银川,1970,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后
1,24395,快驰,男,大专,1997-02-17,2,None,3330,上饶,华中大区,1977,仓库管理部,仓储部,供应链中心,杭州小电科技股份有限公司,None,95后
2,24394,甄帅,男,高中,1995-09-02,2,None,2108,重庆BDM_欧成,重庆二区,1966,重昆分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
3,24393,甄丑,男,大专,1995-07-15,2,None,1274,重庆BDM_风女,重庆二区,1966,重昆分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
4,24392,空尘,男,大专,1997-06-20,2,None,2109,包头小分队BDM_破敌,山西外区,2719,山西分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
5,24391,栩文,男,大专,1997-09-16,2,None,1153,沈阳BDM_涂云,沈阳,953,黑吉辽分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
6,24390,半天,男,大专,1999-04-01,2,None,3321,兰州BDM_二细,兰州二区,3341,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后
7,24389,夜里,男,高中,1995-03-20,2,None,3154,成都BDM_眸光,成都三区,2393,成都分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
8,24388,万槐,男,本科,1996-09-29,2,None,2612,成都BDM_狄云,成都四区,2611,成都分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
9,24387,不名,男,大专,1997-07-07,2,None,2456,东莞BDM_金昊,东莞二区,2547,华南区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后


### 数据探索

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns

user.groupby('性别').size()


性别
1    4186
2     921
dtype: int64

In [121]:
biz_sex_pie=user[ (user['部门名称-2']=='商务发展部') | (user['部门名称-3']=='商务发展部')| (user['部门名称-4']=='商务发展部')| (user['部门名称-5']=='商务发展部')| (user['部门名称-6']=='商务发展部')| (user['部门名称-7']=='商务发展部')].groupby('性别').size()

biz_sex_pie

性别
女     481
男    3554
dtype: int64

In [122]:
doy_sex_pie=user[ (user['部门名称-2']=='产品研发中心') | (user['部门名称-3']=='产品研发中心')| (user['部门名称-4']=='产品研发中心')| (user['部门名称-5']=='产品研发中心')| (user['部门名称-6']=='产品研发中心')| (user['部门名称-7']=='产品研发中心')].groupby('性别').size()
doy_sex_pie

性别
女     37
男    186
dtype: int64

In [123]:
user[ (user['部门名称-2']=='测试质量部') | (user['部门名称-3']=='测试质量部')| (user['部门名称-4']=='测试质量部')| (user['部门名称-5']=='测试质量部')| (user['部门名称-6']=='测试质量部')| (user['部门名称-7']=='测试质量部')].groupby('性别').size()

性别
女    10
男    15
dtype: int64

In [73]:
doy_pie=user[ (user['部门名称-2']=='产品研发中心') | (user['部门名称-3']=='产品研发中心')| (user['部门名称-4']=='产品研发中心')| (user['部门名称-5']=='产品研发中心')| (user['部门名称-6']=='产品研发中心')| (user['部门名称-7']=='产品研发中心')].groupby('学历').size()
doy_pie

学历
中专         1
大专        31
本科       168
硕士研究生     22
高中         1
dtype: int64

In [70]:
biz_pie=user[ (user['部门名称-2']=='商务发展部') | (user['部门名称-3']=='商务发展部')| (user['部门名称-4']=='商务发展部')| (user['部门名称-5']=='商务发展部')| (user['部门名称-6']=='商务发展部')| (user['部门名称-7']=='商务发展部')].groupby('学历').size()

biz_pie

学历
中专        482
初中         32
大专       2387
小学          1
本科        663
硕士研究生       2
高中        469
dtype: int64

In [46]:
user.groupby('学历').size()

学历
MBA         4
中专        537
初中         38
大专       2770
小学          1
本科       1177
硕士研究生      44
高中        536
dtype: int64

In [128]:
age_pie=user.groupby('年龄段').size()
age_pie

年龄段
70后        18
80-85     148
85-90     864
90-95    2352
95后      1722
dtype: int64

In [61]:
a=user.groupby('学历').size()
type(a)
a.index.tolist()
a.values.tolist()

[4, 537, 38, 2770, 1, 1177, 44, 536]

### 数据展现

In [66]:
def user_pie(title_str,lables_str,values_str):
    import plotly.offline as py
    from plotly.graph_objs import Scatter, Layout
    import plotly.graph_objs as go

    py.init_notebook_mode(connected=True)

    pyplt = py.offline.plot
    labels = lables_str
    values = values_str
    trace = [go.Pie(labels=labels, values=values)]
    layout = go.Layout(
        title = title_str,
    )
    figure = go.Figure(data = trace, layout = layout)
    py.iplot(figure)


In [130]:
user_pie('小电整体学历分布比例',a.index.tolist(),a.values.tolist())

In [71]:
user_pie('小电商务发展部学历分布比例',biz_pie.index.tolist(),biz_pie.values.tolist())

In [74]:
user_pie('小电产品研发部学历分布比例',doy_pie.index.tolist(),doy_pie.values.tolist())

In [124]:
user_pie('小电产品研发部性别分布比例',doy_sex_pie.index.tolist(),doy_sex_pie.values.tolist())

In [125]:
user_pie('小电商务发展部性别分布比例',biz_sex_pie.index.tolist(),biz_sex_pie.values.tolist())

In [129]:
user_pie('小电年龄段分布',age_pie.index.tolist(),age_pie.values.tolist())

### 报告撰写

#### 学历
- 小电整体学历大专，本科是主流
- 但是部门之间差异比较大
  1. 在商务发展部，大专是主流
  2. 在产品研发部，本科和硕士是主流
#### 性别
  - 小电是个男多女少的公司，想找妹子的需要避开
#### 年龄
  - 小电整体年龄结构比较年轻，90后是绝对主流

In [10]:
user.head(10)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7,年龄段
0,24397,欧科,男,大专,1996-02-25,2,None,1999,银川BDM_丽莎,银川,1970,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后
1,24395,快驰,男,大专,1997-02-17,2,None,3330,上饶,华中大区,1977,仓库管理部,仓储部,供应链中心,杭州小电科技股份有限公司,None,95后
2,24394,甄帅,男,高中,1995-09-02,2,None,2108,重庆BDM_欧成,重庆二区,1966,重昆分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
3,24393,甄丑,男,大专,1995-07-15,2,None,1274,重庆BDM_风女,重庆二区,1966,重昆分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
4,24392,空尘,男,大专,1997-06-20,2,None,2109,包头小分队BDM_破敌,山西外区,2719,山西分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
5,24391,栩文,男,大专,1997-09-16,2,None,1153,沈阳BDM_涂云,沈阳,953,黑吉辽分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
6,24390,半天,男,大专,1999-04-01,2,None,3321,兰州BDM_二细,兰州二区,3341,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后
7,24389,夜里,男,高中,1995-03-20,2,None,3154,成都BDM_眸光,成都三区,2393,成都分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
8,24388,万槐,男,本科,1996-09-29,2,None,2612,成都BDM_狄云,成都四区,2611,成都分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
9,24387,不名,男,大专,1997-07-07,2,None,2456,东莞BDM_金昊,东莞二区,2547,华南区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后


In [17]:
import datetime

age = datetime.date.today().year - 1997
age

23

In [20]:
def get_age(birth_date):
    age = datetime.date.today().year - birth_date.year
    return age


In [21]:
user['年龄']=user.apply(lambda x: get_age(x['生日']),axis=1)


In [24]:
user.head(30)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7,年龄段,年龄
0,24397,欧科,男,大专,1996-02-25,2,None,1999,银川BDM_丽莎,银川,1970,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后,24
1,24395,快驰,男,大专,1997-02-17,2,None,3330,上饶,华中大区,1977,仓库管理部,仓储部,供应链中心,杭州小电科技股份有限公司,None,95后,23
2,24394,甄帅,男,高中,1995-09-02,2,None,2108,重庆BDM_欧成,重庆二区,1966,重昆分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后,25
3,24393,甄丑,男,大专,1995-07-15,2,None,1274,重庆BDM_风女,重庆二区,1966,重昆分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后,25
4,24392,空尘,男,大专,1997-06-20,2,None,2109,包头小分队BDM_破敌,山西外区,2719,山西分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后,23
5,24391,栩文,男,大专,1997-09-16,2,None,1153,沈阳BDM_涂云,沈阳,953,黑吉辽分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后,23
6,24390,半天,男,大专,1999-04-01,2,None,3321,兰州BDM_二细,兰州二区,3341,西北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后,21
7,24389,夜里,男,高中,1995-03-20,2,None,3154,成都BDM_眸光,成都三区,2393,成都分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后,25
8,24388,万槐,男,本科,1996-09-29,2,None,2612,成都BDM_狄云,成都四区,2611,成都分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后,24
9,24387,不名,男,大专,1997-07-07,2,None,2456,东莞BDM_金昊,东莞二区,2547,华南区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,95后,23


In [23]:
user.describe()

,用户id,级别,部门id-1,部门id-2,年龄
count,5142.00000,5142.000000,5142.000000,5142.000000,5142.000000
mean,16844.48522,3.311746,2087.041813,1669.172695,27.413847
std,6601.70899,2.665173,800.640270,1005.518291,3.881771
min,1.00000,1.000000,4.000000,1.000000,18.000000
25%,12351.25000,2.000000,1535.000000,749.000000,25.000000
50%,18934.00000,2.000000,2185.000000,1767.000000,27.000000
75%,22347.50000,3.000000,2766.000000,2547.000000,30.000000
max,24397.00000,20.000000,3349.000000,3341.000000,51.000000
